### 1. import

In [ ]:
import sys
package_paths = [
    './yolocode/',
]
for pth in package_paths:
    sys.path.append(pth)
    
import re
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
# add path 
import colorsys
from PIL import Image, ImageDraw, ImageFont
import random
from yolo3.model import preprocess_true_boxes, yolo_body, tiny_yolo_body, yolo_loss, yolo_correct_boxes
import PIL
import torch
#!pip install torchmetrics
import torchmetrics.detection.map as MeanAveragePrecision
import shutil
from pprint import pprint

In [ ]:
from yolo3.utils import *

### 2. Function for handling data for active learning

In [26]:
# 1
def make_annotation_txtfile():
    '''This func read image_annotations.csv ['id','file_name','height','width','annotations']
        and split data into unlable pool and test data
    '''
    # select nrows from csv file - number of images in dataset for active learning
    image_annotations = pd.read_csv("./coco_dataset/image_annotations.csv", nrows=10000)
    print('image_annotations',image_annotations.size)
    
    file = open("coco_annotation.txt","a")
    for index, row in image_annotations.iterrows():
        a = row["annotations"]
        a = re.sub(r" ", "", a)
        a = re.sub(r"],", " ", a)
        a = re.sub(r"\[", "", a)
        a = re.sub(r"\]", "", a)
        if a != "":
            #file_path = os.path.join(path, row["file_name"])
            file.writelines(row["file_name"] + " " + a + "\n")
            
    # split into train and test
    file = open("coco_annotation.txt","r")
    a = len(file.readlines())
    train_num = int(a*0.8) ## Split data into 80 and 20 percent of select data 
    test_num = a - train_num
    file.close()
    train_file = open("coco_unlabel_pool.txt", "a")
    test_file = open("coco_test.txt", "a")
    with open("coco_annotation.txt","r") as file:
        for i, line in enumerate(file):
            if i <= train_num:
                train_file.writelines(line)
            else:
                test_file.writelines(line)
    train_file.close()
    test_file.close()
    file.close()

In [27]:
# 2
# select top n training data
def select_train_data(n=100, append=False):
    ''' This function select training images from unlabel pool.  '''
    temp_file = open("temp.txt","a+")
    if append:
        train_file = open("coco_train.txt","a+")
    else:
        train_file = open("coco_train.txt","w")
    pool_file = open("coco_unlabel_pool.txt","r")
    a  = pool_file.readlines()
    
    for i in range(n):
        train_file.writelines(a[i])
    for i in range(n,len(a)):
        temp_file.writelines(a[i])

    pool_file.close()
    temp_file.close() 
    train_file.close()
    if os.path.exists("coco_unlabel_pool.txt"):
        os.remove("coco_unlabel_pool.txt")
        os.rename('temp.txt', 'coco_unlabel_pool.txt')

In [28]:
# 3
def move_to_train(selected):
    temp_file = open("temp.txt","a+")
    train_file = open("coco_train.txt","a+")
    pool_file = open("coco_unlabel_pool.txt","r")
    lines  = pool_file.readlines()
    
    for line in lines:
        name = line.split()[0]
        if name in selected:
            train_file.writelines(line)
        else:
            temp_file.writelines(line)

    pool_file.close()
    temp_file.close() 
    train_file.close()
    if os.path.exists("coco_unlabel_pool.txt"):
        os.remove("coco_unlabel_pool.txt")
        os.rename('temp.txt', 'coco_unlabel_pool.txt')

In [29]:
# 4
def get_unlabel_images():
    """ This function return names of images in unlable pool. """
    pool_file = open("coco_unlabel_pool.txt","r")
    lines  = pool_file.readlines()
    unlable_images = []
    for i in lines:
        unlable_images.append(i.split()[0])
        
    return unlable_images

# 5
def annotated_images(model_path = "./yolocode/model_data/yolo_weights.h5",n = 1000, score_type = 'normal'):
    """ This function make prediction on all the images in unlabel pool and
    select training images form unlabel pool based of images scores """
    # load trained model
    model = load_trained_model(input_shape=(416,416), num_classes=80, load_pretrained=True, freeze_body=2,
            weights_path=model_path)

    unlable_images = get_unlabel_images()
    
    # calculate score for images
    images_score = np.empty((0))
    iter = 0
    for i in chunks(unlable_images,10):
        if iter*10 >= n:
            break
        if iter%10==0:
            print('progress iteration ', iter)
        iter = iter + 1
        batch = np.empty((0,416,416,3))
        for img in  i:
            image = read_image(img)
            image = preprocess_image(image)
            batch = np.append(batch,image,axis=0)
        score, prob_score = get_score(batch,model)
#         print('prob_score', prob_score)
        if score_type == 'prob':
            images_score = np.append(images_score,prob_score)
        elif score_type == 'normal':
            images_score = np.append(images_score,score)
    
     # select top n images
    top_score = (images_score).argsort()[:n]
    selected = []
    for i in top_score:
        selected.append(unlable_images[i])
    move_to_train(selected)   

        

### 3. Training yolo function

In [30]:
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

from yolo3.model import preprocess_true_boxes, yolo_body, tiny_yolo_body, yolo_loss
from yolo3.utils import get_random_data

# 1
def get_classes(classes_path="./yolocode/model_data/coco_classes.txt"):
    '''loads the classes'''
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

# 2
def get_anchors(anchors_path="./yolocode/model_data/yolo_anchors.txt"):
    '''loads the anchors from a file'''
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)
# 3
def create_model(input_shape, anchors, num_classes, load_pretrained=True, freeze_body=0,
            weights_path='./yolocode/model_data/yolo_weights.h5'):
    '''create the training model'''
    K.clear_session() # get a new session
    image_input = Input(shape=(None, None, 3))
    h, w = input_shape
    num_anchors = len(anchors)

    y_true = [Input(shape=(h//{0:32, 1:16, 2:8}[l], w//{0:32, 1:16, 2:8}[l], \
        num_anchors//3, num_classes+5)) for l in range(3)]
    ## create list of o/p shape - 
    ## shape=(None, 13, 13, num_anchors/3 , num_classes+5); 
    ## shape=(None, 26, 26, num_anchors/3, num_classes+5); 
    ## shape=(None, 52, 52, num_anchors/3, num_classes+5)
    ## +5 for [xmin,ymin,xmax,ymax,class] for all classes and all anchore div into 3 
    # load model architecture
    model_body = yolo_body(image_input, num_anchors//3, num_classes)
    print('Create YOLOv3 model with {} anchors and {} classes.'.format(num_anchors, num_classes))
    
    if load_pretrained:
        model_body.load_weights(weights_path, by_name=True, skip_mismatch=True)
        print('Load weights {}.'.format(weights_path))
        if freeze_body in [1, 2]:
            # Freeze darknet53 body or freeze all but 3 output layers.
            num = (185, len(model_body.layers)-3)[freeze_body-1]
            for i in range(num): model_body.layers[i].trainable = False
            print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))

    model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
        arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.5})(
        [*model_body.output, *y_true])
    model = Model([model_body.input, *y_true], model_loss)

    return model

# 4
def data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes):
    '''data generator for fit_generator'''
    n = len(annotation_lines)
    i = 0
    while True:
        image_data = []
        box_data = []
        for b in range(batch_size):
            if i==0:
                np.random.shuffle(annotation_lines)
            image, box = get_random_data(annotation_lines[i], input_shape, random=True)
            image_data.append(image)
            box_data.append(box)
            i = (i+1) % n
        image_data = np.array(image_data)
        box_data = np.array(box_data)
        y_true = preprocess_true_boxes(box_data, input_shape, anchors, num_classes)
        yield [image_data, *y_true], np.zeros(batch_size)

# 5
def data_generator_wrapper(annotation_lines, batch_size, input_shape, anchors, num_classes):
    n = len(annotation_lines)
    if n==0 or batch_size<=0: return None
    return data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes)

# 6
def train(batch = 32,epoch = 50,load_pretrained=False,weights_path="./logs/000/trained_weights.h5"):
    annotation_path = './coco_train.txt'
    log_dir = 'logs/000/'
    classes_path = './yolocode/model_data/coco_classes.txt'
    anchors_path = './yolocode/model_data/yolo_anchors.txt'
    class_names = get_classes(classes_path) ## get list of names
    num_classes = len(class_names)
    anchors = get_anchors(anchors_path) ## get list of 2d lists
  
    input_shape = (416,416) # multiple of 32, hw
  

  
    ## LOAD MODEL
    model = create_model(input_shape, anchors, num_classes, load_pretrained,weights_path) 
    
  
    logging = TensorBoard(log_dir=log_dir)
    checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',monitor='val_loss', save_weights_only=True, save_best_only=True, period=3)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)
  
    # read train annotaion and split 90% train 10% val
    val_split = 0.1
    with open(annotation_path) as f:
        lines = f.readlines()
    np.random.seed(10101)
    np.random.shuffle(lines)
    np.random.seed(None)
    num_val = int(len(lines)*val_split)
    num_train = len(lines) - num_val
  

    if True:
        for i in range(len(model.layers)):
            model.layers[i].trainable = True
        model.compile(optimizer=Adam(learning_rate=1e-4), loss={'yolo_loss': lambda y_true, y_pred: y_pred}) # recompile to apply the change
        #print('Unfreeze all of the layers.')
        batch_size = batch # note that more GPU memory is required after unfreezing the body
        print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
        model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
            steps_per_epoch=max(1, num_train//batch_size),
            validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
            validation_steps=max(1, num_val//batch_size),
            epochs=epoch,
            initial_epoch=0,
            callbacks=[logging, checkpoint, reduce_lr, early_stopping])
        model.save_weights(log_dir + 'trained_weights.h5')
        
    return model

### 4. Yolo untility functions

In [31]:
# preprocess
# 1
def read_image(image_name):
    
    path = "./coco_dataset/images/"
    image = PIL.Image.open(path + image_name)
    return image
    
# 2   
def preprocess_image(image):
    sq_image = letterbox_image(image, (416,416))
    image_data = np.array(sq_image)
    image_data = image_data/255.
    image_data = np.expand_dims(image_data, 0) # (1, 416, 416, 3)
    return image_data

# 3
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [32]:
# 4
def load_trained_model(input_shape=(416,416), num_classes=80, load_pretrained=True, freeze_body=2,
            weights_path='model_data/yolo_weights.h5'):
    '''create the training model'''
    #anchors_path = '../input/yolocode/model_data/yolo_anchors.txt'
    
    anchors = get_anchors()
    
    
    K.clear_session() # get a new session
    image_input = Input(shape=(None, None, 3))
    h, w = input_shape
    num_anchors = len(anchors)

    y_true = [Input(shape=(h//{0:32, 1:16, 2:8}[l], w//{0:32, 1:16, 2:8}[l], \
        num_anchors//3, num_classes+5)) for l in range(3)]
    ## create list of o/p shape - 
    ## shape=(None, 13, 13, num_anchors/3 , num_classes+5); 
    ## shape=(None, 26, 26, num_anchors/3, num_classes+5); 
    ## shape=(None, 52, 52, num_anchors/3, num_classes+5)
    ## +5 for [xmin,ymin,xmax,ymax,class] for all classes and all anchore div into 3 

    # load model architecture
    model_body = yolo_body(image_input, num_anchors//3, num_classes)
    print('Create YOLOv3 model with {} anchors and {} classes.'.format(num_anchors, num_classes))

    # LOAD pretarin model weights
    
    if load_pretrained:
        model_body.load_weights(weights_path, by_name=True, skip_mismatch=True)
        print('Load weights {}.'.format(weights_path))
        if freeze_body in [1, 2]:
            # Freeze darknet53 body or freeze all but 3 output layers.
            num = (185, len(model_body.layers)-3)[freeze_body-1]
            for i in range(num): model_body.layers[i].trainable = False
            print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))

    return model_body

# 5
def get_score(image,yolo_model):
    """ This function calculate score of the image """
    # get data
    anchors = get_anchors()
    class_names = get_classes()
    num_anchors = len(anchors)
    num_classes = len(class_names)
     
    yolo_outputs = yolo_model.predict(image)
    # (1, 13, 13, 255) 
    # (1, 26, 26, 255)
    # (1, 52, 52, 255)
    
    
    score_batch = np.zeros((image.shape[0])) # ((None,416,416,3))
    al_ep_score = np.zeros((image.shape[0]))
    for i in range(len(image)):
        _output = [ np.expand_dims(yolo_outputs[0][i],0), np.expand_dims(yolo_outputs[1][i],0), np.expand_dims(yolo_outputs[2][i],0) ]
        out_boxes, out_scores, out_classes, uncert_all_classes = yolo_eval(_output,anchors,num_classes,
                                                   [416,416],
                                                   max_boxes=20,score_threshold=.6,
                                                   iou_threshold=.5)
        
        # calculating aleatoric and epistemic uncertainities for every image
#         print('out_classes', out_classes)
#         print('out_scores', out_scores)
#         print('uncert', uncert_all_classes)
        u_al = 0
        u_ep = 0
        normalized_scores = out_scores
        if len(out_scores)>0:
            normalized_scores = out_scores/np.max(out_scores)
            normalized_scores = normalized_scores.numpy()
        out_classes = out_classes.numpy()
#         print('normalized', normalized_scores)
        sum_pi_mu = 0
        for c,s in zip(out_classes,normalized_scores):
            for b in uncert_all_classes[c]:
                for u in b:
#                     print('s', s, 'mu', u[0])
                    sum_pi_mu += np.log(s*tf.clip_by_value(u[0], clip_value_min=0.0, clip_value_max=tf.float32.max)+1)
        
        for c,s in zip(out_classes,normalized_scores):
            for b in uncert_all_classes[c]:
                for u in b:
                    mu = u[0]
                    var = u[1]
#                     print('mu', mu, 'var', var)
                    u_al += np.log(s*var+1)
                    u_ep += np.log(s*np.abs(mu - sum_pi_mu)**2+1)
        
        
        score_batch[i] = np.average(out_scores)   
#         print('u_al', u_al, 'u_ep', u_ep)
        al_ep_score[i] = u_al + u_ep
    return score_batch, al_ep_score

In [32]:
#6
def yolo_predict(image, model_path = "./yolocode/model_data/yolo_weights.h5"):
    # preprocess image
    image_data = preprocess_image(image) # (1, 416, 416, 3)
    
    # get data
    anchors = get_anchors()
    class_names = get_classes()
    num_anchors = len(anchors)
    num_classes = len(class_names)
    
    # load model
    yolo_model = load_trained_model(input_shape=(416,416), num_classes=num_classes, load_pretrained=True,freeze_body=2,weights_path=model_path)
    #yolo_model = load_model(model_path, compile=False)
    
    yolo_outputs = yolo_model.predict(image_data)
    # (1, 13, 13, 255)
    # (1, 26, 26, 255)
    # (1, 52, 52, 255)
    
    out_boxes, out_scores, out_classes, _ = yolo_eval(yolo_outputs,anchors,num_classes,[image.size[1],image.size[0]],max_boxes=20,score_threshold=.6,iou_threshold=.5)
    image = draw_boxes(out_boxes, out_scores, out_classes, image)
    return image

In [5]:
# 7
def get_uncert_score(boxes, scores, max_output_size, iou_threshold=0.5, score_threshold=float('-inf'), name=None):
    """ This function calculate uncertanty score before applying nms """
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
    scores = scores
    order = tf.argsort(scores)
    areas = (x2-x1)*(y2-y1)
    keep = []
    nms_idx = []
    uncert = []
    while len(order) > 0:
        idx = order[-1]
        
        keep.append(boxes[idx].numpy())
        nms_idx.append(idx.numpy())
        order = order[:-1]

        xx1 = tf.gather(x1,indices=order,axis=0)
        xx2 = tf.gather(x2,indices=order,axis=0)
        yy1 = tf.gather(y1,indices=order,axis=0)
        yy2 = tf.gather(y2,indices=order,axis=0)
        
        xx1 = tf.math.maximum(xx1,x1[idx])
        yy1 = tf.math.maximum(yy1,y1[idx])
        xx2 = tf.math.minimum(xx2,x2[idx])
        yy2 = tf.math.minimum(yy2,y2[idx])
        
        w = xx2 - xx1
        h = yy2 - yy1
        
        w = tf.clip_by_value(w, clip_value_min=0.0, clip_value_max = tf.float32.max)
        h = tf.clip_by_value(h, clip_value_min=0.0, clip_value_max = tf.float32.max)
        
        inter = w*h
        
        rem_areas = tf.gather(areas,indices=order,axis=0)
        
        union = (rem_areas - inter) + areas[idx]
        
        IoU = inter/union
        
#         mask = IoU < iou_threshold
#         order = order[mask]
        mask = IoU < 0.5
        inv_mask = IoU > 0.5
        
        cluster_idx = order[inv_mask]
        cluster_idx = list(cluster_idx.numpy())
        cluster_idx.append(idx.numpy())
#         tf.stack([cluster_idx.numpy(), [idx.numpy()]], axis=0)
        order = order[mask]
        
        
        
        x1_ = [x1[i].numpy() for i in cluster_idx]
        y1_ = [y1[i].numpy() for i in cluster_idx]
        x2_ = [x2[i].numpy() for i in cluster_idx]
        y2_ = [y2[i].numpy() for i in cluster_idx]
        
        x1_ = np.array(x1_)
        y1_ = np.array(y1_)
        x2_ = np.array(x2_)
        y2_ = np.array(y2_)
        
        x1_mu = x1_.mean() 
        y1_mu = y1_.mean()
        x2_mu = x2_.mean()
        y2_mu = y2_.mean()
        
        x1_var = x1_.var() 
        y1_var = y1_.var()
        x2_var = x2_.var()
        y2_var = y2_.var()
        
        
        uncert.append([[x1_mu, x1_var], [y1_mu, y1_var], [x2_mu, x2_var], [y2_mu, y2_var]])
    
    return tf.convert_to_tensor(keep,dtype=tf.float32), tf.convert_to_tensor(nms_idx,dtype=tf.int32), uncert


In [35]:
# 8
def yolo_eval(yolo_outputs,
              anchors,
              num_classes,
              image_shape,
              max_boxes=20,
              score_threshold=.6,
              iou_threshold=.5):
    """Evaluate YOLO model on given input and return filtered boxes."""
    num_layers = len(yolo_outputs)
       
    anchor_mask = [[6,7,8], [3,4,5], [0,1,2]] if num_layers==3 else [[3,4,5], [1,2,3]] # default setting
    input_shape = K.shape(yolo_outputs[0])[1:3] * 32
    boxes = []
    box_scores = []
    for l in range(num_layers):
        _boxes, _box_scores = yolo_boxes_and_scores(yolo_outputs[l],
            anchors[anchor_mask[l]], num_classes, input_shape, image_shape)
        boxes.append(_boxes)
        box_scores.append(_box_scores)
    boxes = K.concatenate(boxes, axis=0)
    box_scores = K.concatenate(box_scores, axis=0)

    mask = box_scores >= score_threshold
    max_boxes_tensor = K.constant(max_boxes, dtype='int32')
    boxes_ = []
    scores_ = []
    classes_ = []
    uncert_all_classes = []
    for c in range(num_classes):
        # TODO: use keras backend instead of tf.
        class_boxes = tf.boolean_mask(boxes, mask[:, c])
        class_box_scores = tf.boolean_mask(box_scores[:, c], mask[:, c])
        nms_index = tf.image.non_max_suppression(
            class_boxes, class_box_scores, max_boxes_tensor, iou_threshold=iou_threshold)
        mynms_out, mynms_index, uncert = get_uncert_score(class_boxes, class_box_scores, max_boxes_tensor, iou_threshold=iou_threshold)
        uncert_all_classes.append(uncert)
        class_boxes = K.gather(class_boxes, nms_index)
        class_box_scores = K.gather(class_box_scores, nms_index)
        classes = K.ones_like(class_box_scores, 'int32') * c
        boxes_.append(class_boxes)
        scores_.append(class_box_scores)
        classes_.append(classes)
    boxes_ = K.concatenate(boxes_, axis=0)
    scores_ = K.concatenate(scores_, axis=0)
    classes_ = K.concatenate(classes_, axis=0)

    return boxes_, scores_, classes_, uncert_all_classes

# 9
def yolo_boxes_and_scores(feats, anchors, num_classes, input_shape, image_shape):
    '''Process Conv layer output'''
    box_xy, box_wh, box_confidence, box_class_probs = yolo_head(feats,
        anchors, num_classes, input_shape)    
    boxes = yolo_correct_boxes(box_xy, box_wh, input_shape, image_shape)
    boxes = K.reshape(boxes, [-1, 4])
    box_scores = box_confidence * box_class_probs
    box_scores = K.reshape(box_scores, [-1, num_classes])
    return boxes, box_scores

# 10
def yolo_head(feats, anchors, num_classes, input_shape, calc_loss=False):
    """Convert final layer features to bounding box parameters."""
    num_anchors = len(anchors)
    # Reshape to batch, height, width, num_anchors, box_params.
    anchors_tensor = K.reshape(K.constant(anchors), [1, 1, 1, num_anchors, 2])

    grid_shape = K.shape(feats)[1:3] # height, width
    grid_y = K.tile(K.reshape(K.arange(0, stop=grid_shape[0]), [-1, 1, 1, 1]),
        [1, grid_shape[1], 1, 1])
    grid_x = K.tile(K.reshape(K.arange(0, stop=grid_shape[1]), [1, -1, 1, 1]),
        [grid_shape[0], 1, 1, 1])
    grid = K.concatenate([grid_x, grid_y])
    grid = K.cast(grid, np.float32)

    feats = K.reshape(
        feats, [-1, grid_shape[0], grid_shape[1], num_anchors, num_classes + 5])
    
    
    # Adjust preditions to each spatial grid point and anchor size.
    box_xy = (K.sigmoid(feats[..., :2]) + grid) / K.cast(grid_shape[::-1], K.dtype(feats))
    box_wh = K.exp(feats[..., 2:4]) * anchors_tensor / K.cast(input_shape[::-1], K.dtype(feats))
    box_confidence = K.sigmoid(feats[..., 4:5])
    box_class_probs = K.sigmoid(feats[..., 5:])

    if calc_loss == True:
        return grid, feats, box_xy, box_wh
    
    return box_xy, box_wh, box_confidence, box_class_probs

# 11
def draw_boxes(out_boxes, out_scores, out_classes, image):
    class_names = get_classes()
    font = ImageFont.load_default()
    thickness = (image.size[0] + image.size[1]) // 300
    colors = get_colors_for_classes(len(class_names))
    # out_classes
    for i, c in reversed(list(enumerate(out_classes))):
        predicted_class = class_names[c]
        box = out_boxes[i]
        score = out_scores[i]
    
        label = '{} {:.2f}'.format(predicted_class, score)
        draw = ImageDraw.Draw(image)
        label_size = draw.textsize(label, font)
    
        top, left, bottom, right = box
        top = max(0, np.floor(top + 0.5).astype('int32'))
        left = max(0, np.floor(left + 0.5).astype('int32'))
        bottom = min(image.size[1], np.floor(bottom + 0.5).astype('int32'))
        right = min(image.size[0], np.floor(right + 0.5).astype('int32'))
        print(label, (left, top), (right, bottom))
    
        if top - label_size[1] >= 0:
            text_origin = np.array([left, top - label_size[1]])
        else:
            text_origin = np.array([left, top + 1])
    
        
        for i in range(thickness):
            draw.rectangle(
                [left + i, top + i, right - i, bottom - i],
                outline=colors[c])
        draw.rectangle(
            [tuple(text_origin), tuple(text_origin + label_size)],
            fill=colors[c])
        draw.text(list(text_origin), label, fill=(0, 0, 0), font=font)
        del draw
    
    return image

# 12
def get_colors_for_classes(num_classes):
    """Return list of random colors for number of classes given."""
    # Use previously generated colors if num_classes is the same.
    if (hasattr(get_colors_for_classes, "colors") and
            len(get_colors_for_classes.colors) == num_classes):
        return get_colors_for_classes.colors

    hsv_tuples = [(x / num_classes, 1., 1.) for x in range(num_classes)]
    colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
    colors = list(
        map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)),
            colors))
    random.seed(10101)  # Fixed seed for consistent colors across runs.
    random.shuffle(colors)  # Shuffle colors to decorrelate adjacent classes.
    random.seed(None)  # Reset seed to default.
    get_colors_for_classes.colors = colors  # Save colors for future calls.
    return colors

#### yolo test functions

In [36]:
# 13
def get_test():
    # list images names And its annotation
    test_file = open("./coco_test.txt","r")
    lines  = test_file.readlines()
    images = []
    labels = []
    for i in lines:
        images.append(i.split()[0])
        labels.append(i.split()[1:])
    return images, labels

#14
def yolo_test(model_path="./yolocode/model_data/yolo_weights.h5"):
    """ This function read images from test set and make prediction it return ground truth and prediction """
    TESTED = 5000
    
    test_images, test_labels = get_test()
    # ['5,35,475,379,48', '0,287,342,426,lable']
    # load model
    yolo_model = load_model(model_path, compile=False)
    
    # get data
    anchors = get_anchors()
    class_names = get_classes()
    num_anchors = len(anchors)
    num_classes = len(class_names)
    
    ann = []
    det = []
    for i in range(len(test_images)):
        print("Testing : " + str(i) + "/" + str(TESTED), end="\r")
        img = test_images[i]
        labels = test_labels[i]
        
        # read image
        image = read_image(img)

        
        ### Create annotation formate
        for k in range(len(labels)):            
            labels[k] = labels[k].split(",")
            labels[k] = [int(j) for j in labels[k]] # create int list
        labels = np.array(labels)
        labels[:, [2, 1]] = labels[:, [1, 2]]
        labels = np.insert(labels, 0,0, axis=1)
        
        labels = labels[:, [0,5,1,2,3,4]]
        
        df = pd.DataFrame(labels,columns =['name','class','xmin','xmax','ymin','ymax'])
        df = df.replace({'name': 0}, img)
        df = df.replace({"class": dict(zip(range(len(class_names)), class_names))})
        df['xmin'] = df['xmin'].div(image.size[0])
        df['xmax'] = df['xmax'].div(image.size[0])
        df['ymin'] = df['ymin'].div(image.size[1])
        df['ymax'] = df['ymax'].div(image.size[1])
        ann.append(df)
        # preprocess image
        image_data = preprocess_image(image) # (1, 416, 416, 3)
        
        yolo_outputs = yolo_model.predict(image_data)
        # (1, 13, 13, 255)     # (1, 26, 26, 255)      # (1, 52, 52, 255)
        
        out_boxes, out_scores, out_classes, _ = yolo_eval(yolo_outputs,anchors,num_classes,[image.size[1],image.size[0]],max_boxes=20,score_threshold=.6,iou_threshold=.5)
        # TODO: using labels and (out_boxes, out_classes) calcuate map
        
        # create output labels formate
        out_boxes = np.array(out_boxes)
        # correct boxes
        for j in range(len(out_boxes)):
            out_boxes[j][0] =  max(0, out_boxes[j][0])
            out_boxes[j][1] =  max(0, out_boxes[j][1])
            out_boxes[j][2] =  min(image.size[1], out_boxes[j][2])
            out_boxes[j][3] =  min(image.size[0], out_boxes[j][3])
        out_boxes[:, [2, 1]] = out_boxes[:, [1, 2]] # swap xmin,xmax,ymin,ymax
        out_boxes = np.insert(out_boxes, 0,out_scores, axis=1) # add score
        out_boxes = np.insert(out_boxes, 0,out_classes, axis=1) # add class
        out_boxes = np.insert(out_boxes, 0,0, axis=1) # add 0 for img name
        
        df2 = pd.DataFrame(out_boxes,columns =['name','class','score','xmin','xmax','ymin','ymax'])
        df2 = df2.replace({'name': 0}, img)
        df2 = df2.replace({"class": dict(zip(range(len(class_names)), class_names))})
        df2['xmin'] = df2['xmin'].div(image.size[0])
        df2['xmax'] = df2['xmax'].div(image.size[0])
        df2['ymin'] = df2['ymin'].div(image.size[1])
        df2['ymax'] = df2['ymax'].div(image.size[1])
        det.append(df2)
        if i == TESTED:
            break
    ann = pd.concat(ann)
    det = pd.concat(det)
    return ann, det
        

### 5. mAP Score functions

In [43]:
def getDataForMap(model_path):
    """ This function calculate mAP for test data given model path """
    test_images, test_labels = get_test()
    
    # get targets
    target = []
    preds = []
    
    # get data
    anchors = get_anchors()
    class_names = get_classes()
    num_anchors = len(anchors)
    num_classes = len(class_names)
    
    # load model
    yolo_model = load_trained_model(input_shape=(416,416), num_classes=num_classes, load_pretrained=True,freeze_body=2,weights_path=model_path)
    #yolo_model = load_model(model_path, compile=False)
    iii = 0
    
    
    print("total test:" , len(test_labels))
    for a in test_labels:
        iii = iii +1
        
#         if iii == 100:
#             break
        print(iii)
        boxes = []
        labels = []
        for b in a:
            c = b.split(",")
            c = [int(i) for i in c]
            
            #bbox = [c[0],c[1],c[0]+c[2],c[1]+c[3]]
            
            boxes.append(c[:-1])
            labels.append(c[-1])
              
        #labels = [i-1 for i in labels]
        target.append(
        dict(
            boxes=torch.tensor(boxes),
            labels=torch.tensor(labels),
            )
        )
    
    iii = 0
    print(len(test_images))
    for names in test_images:
        iii = iii +1
        
#         if iii == 100:
#             break
        print(iii)
        image = read_image(names)
        image_data = preprocess_image(image) # (1, 416, 416, 3)
        
        
        
        yolo_outputs = yolo_model.predict(image_data)
        
        out_boxes, out_scores, out_classes, _ = yolo_eval(yolo_outputs,anchors,num_classes,[image.size[1],image.size[0]],max_boxes=20,score_threshold=.6,iou_threshold=.5)
        
        out_boxes =  out_boxes.numpy()
        
        for i in range(len(out_boxes)):
            out_boxes[i][0] = max(0, np.floor(out_boxes[i][0] + 0.5).astype('int32'))
            out_boxes[i][1] = max(0, np.floor(out_boxes[i][1] + 0.5).astype('int32'))
            out_boxes[i][2] = min(image.size[1], np.floor(out_boxes[i][2] + 0.5).astype('int32'))
            out_boxes[i][3] = min(image.size[0], np.floor(out_boxes[i][3] + 0.5).astype('int32'))
        
        out_boxes = out_boxes[:, [1, 0, 3, 2]]
        
        
        preds.append(dict(
            boxes=torch.tensor(out_boxes),
            scores=torch.tensor(out_scores.numpy()),
            labels=torch.tensor(out_classes.numpy()),
            ))
    
        
    return preds, target

### 6. Active learning loop

#### 6.1 Setting data

In [38]:
## Setting data
## making annotation files
make_annotation_txtfile()
select_train_data(100) # select  init data for training

#### 6.2 Loop

In [ ]:
BATCH_SIZE = 16
EPOCHS = 5
log_dir = 'logs/000/'

# train intial model
train(batch = BATCH_SIZE,epoch = EPOCHS, load_pretrained=True,weights_path="./yolocode/model_data/yolo_weights.h5")
AL_map = []

# active learning loop
for i in range(100):
    # select new images for training 
    print('Active Learning Iteration ',i)
    ## change score_type to select scoring function normal / prob (for uncertanty based scoring function)
    ## annotate image based on previously trained model
    annotated_images(model_path = "./logs/000/trained_weights.h5",n=30,score_type = 'normal')
    
    # train new model with new training images selected
    train(batch = BATCH_SIZE,epoch = EPOCHS,load_pretrained=True,weights_path="./yolocode/model_data/yolo_weights.h5" )
    shutil.copyfile(log_dir + "/trained_weights.h5", log_dir + "/trained_weights_"+str(i)+".h5")
    
    # calculate mAP score
    preds, target = getDataForMap(log_dir + "/trained_weights_"+str(i)+".h5")
    metric = MeanAveragePrecision.MAP()
    metric.update(preds, target)
    pprint(metric.compute())
    AL_map.append(metric.compute())